In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
torch.manual_seed(0)

import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
from torch.utils.data import random_split


import torchattacks

In [3]:
from cifar_model_arch.densenet import DenseNet121
from cifar_model_arch.efficientnet import EfficientNetB0
from cifar_model_arch.resnet import ResNet18
from cifar_model_arch.swin import swin_s
from cifar_model_arch.vit import ViT

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_transform = transforms.Compose([
    transforms.ToTensor(),
])

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100,
                                          shuffle=True, num_workers=8)

Files already downloaded and verified


In [5]:
save_path = 'save_path_with_a_lot_of_memory'

In [6]:
vit_params = {
    'image_size' : 32,
    'patch_size' : 4,
    'num_classes' : 10,
    'dim' : 384,
    'depth' : 7,
    'heads' : 12,
    'mlp_dim' : 384*3,
    'dropout' : 0.1,
    'emb_dropout' : 0
}

EfficientNetB0_params = {}
DenseNet121_params = {}
Resnet18_params = {}
Swin_params = {'num_classes':10}

params = {'ViT': vit_params, 'ViT_copy': vit_params,
          'DenseNet121': DenseNet121_params, 'DenseNet121_copy': DenseNet121_params,
          'EfficientNetB0': EfficientNetB0_params, 'EfficientNetB0_copy': EfficientNetB0_params,
          'ResNet18': Resnet18_params, 'ResNet18_copy': Resnet18_params,
          'swin_s': Swin_params, 'swin_s_copy': Swin_params}



model_paths = { 'ViT': './models/cifar10/ViT_0.ckpt',
               'ViT_copy': './models/cifar10/ViT_1.ckpt',
                'DenseNet121':'./models/cifar10/DenseNet121_0.ckpt',
                'DenseNet121_copy':'./models/cifar10/DenseNet121_1.ckpt',
                'EfficientNetB0': './models/cifar10/EfficientNetB0_0.ckpt',
                'EfficientNetB0_copy': './models/cifar10/EfficientNetB0_1.ckpt',
                'ResNet18': './models/cifar10/resnet18_0.ckpt',
                'ResNet18_copy': './models/cifar10/resnet18_1.ckpt',
                'swin_s': './models/cifar10/Swin_0.ckpt',
                'swin_s_copy': './models/cifar10/Swin_1.ckpt'}
                    

models = {'ViT': ViT, 'ViT_copy': ViT, 'DenseNet121': DenseNet121, 'DenseNet121_copy': DenseNet121,
          'EfficientNetB0':EfficientNetB0, 'EfficientNetB0_copy':EfficientNetB0,
          'ResNet18': ResNet18, 'ResNet18_copy': ResNet18,
          'swin_s': swin_s, 'swin_s_copy': swin_s}

model_names = ['ViT', 'DenseNet121', 'EfficientNetB0', 'ResNet18', 'swin_s', 
              'ViT_copy', 'DenseNet121_copy', 'EfficientNetB0_copy', 'ResNet18_copy', 'swin_s_copy']


eps = 4/255
alpha = 1/255
iters = 40
attack = 'PGD'

In [7]:
import os
os.mkdir(os.path.join(save_path,
                      f'adv_samples/PGD/cifar/pertubation_ranking'))

FileExistsError: [Errno 17] File exists: '/dt/yisroel/Chaimeleon/Transferability/adv_samples/PGD/cifar/pertubation_ranking'

In [19]:
# for model_name in model_names:
#     if '_copy'  in model_name:
#         continue
#     m = models[model_name](**params[model_name]).to(device)
#     m.load_state_dict(torch.load(model_paths[model_name])['net'])
#     print(f'loaded {model_paths[model_name]}')
#     atk = torchattacks.PGD(m,random_start=False, eps=eps,alpha=alpha,steps=iters)
#     # atk = torchattacks.MIFGSM(m, eps=eps, steps=iters, decay=0.9)
#     atk = torchattacks.FGSM(m, eps=eps)
    
#     true_labels = []
#     adv_inputs = []
#     sample_indexes = []
#     sample_num = 0

#     for batch, labels in test_loader:
#         batch, labels = batch.to(device), labels.to(device)
#         sample_index = torch.arange(sample_num, sample_num+labels.size(0))
#         adv_samples = atk(batch, labels)
#         pred_labels = m(adv_samples).argmax(dim=1)
#         mask = pred_labels.eq(labels)
#         if mask.all():
#             continue
#         else:
#             adv_samples = adv_samples[~mask]
#             labels = labels[~mask]
#             sample_index = sample_index[~mask.cpu()]
#             true_labels.append(labels.to('cpu'))
#             adv_inputs.append(adv_samples.to('cpu'))
#             sample_indexes.append(sample_index)
#         sample_num += labels.size(0)

        
#     true_labels = torch.cat(true_labels)
#     adv_inputs = torch.cat(adv_inputs, dim=0)
#     sample_indexes = torch.cat(sample_indexes)

#     torch.save(true_labels, f'./adv_samples/{attack}/cifar10/{model_name}_labels.ckpt')
#     torch.save(adv_inputs, f'./adv_samples/{attack}/cifar10/{model_name}_samples.ckpt')
#     torch.save(sample_indexes, f'./adv_samples/{attack}/cifar10/{model_name}_index.ckpt')

#     del m
#     del atk
    
batch, labels = next(iter(test_loader))
batch, labels = batch.to(device), labels.to(device)

for model_name in model_names:
    m = models[model_name](**params[model_name]).to(device)
    m.load_state_dict(torch.load(model_paths[model_name])['net'])
    print(f'loaded {model_paths[model_name]}')
    
    m.eval()
    atk = torchattacks.PGD(m, random_start=True, 
                           eps=eps,alpha=alpha,steps=iters)

    for run in range(0, 100):
        true_labels = labels.to('cpu')
        adv_samples = atk(batch, labels).to('cpu')
    
        torch.save(true_labels, os.path.join(save_path,
                                             f'adv_samples/PGD/cifar/pertubation_ranking/{model_name}_labels_run_{run}.ckpt'))
        torch.save(adv_samples,  os.path.join(save_path,
                                             f'adv_samples/PGD/cifar/pertubation_ranking/{model_name}_samples_run_{run}.ckpt'))
    del atk, m
      
        
    

loaded ./models/cifar10/ViT_0.ckpt
loaded ./models/cifar10/DenseNet121_0.ckpt
loaded ./models/cifar10/EfficientNetB0_0.ckpt
loaded ./models/cifar10/resnet18_0.ckpt
loaded ./models/cifar10/Swin_0.ckpt
loaded ./models/cifar10/ViT_1.ckpt
loaded ./models/cifar10/DenseNet121_1.ckpt
loaded ./models/cifar10/EfficientNetB0_1.ckpt
loaded ./models/cifar10/resnet18_1.ckpt
loaded ./models/cifar10/Swin_1.ckpt


In [8]:
other_list = {'ResNet18': ['ViT', 'ResNet18_copy', 'DenseNet121', 'EfficientNetB0', 'swin_s'],
              'swin_s': ['ViT', 'swin_s_copy', 'DenseNet121', 'EfficientNetB0', 'ResNet18'],
              'ViT': ['DenseNet121', 'ViT_copy', 'EfficientNetB0', 'ResNet18', 'swin_s'],
              'DenseNet121': ['ViT','DenseNet121_copy', 'EfficientNetB0', 'ResNet18', 'swin_s'],
              'EfficientNetB0': ['ViT', 'EfficientNetB0_copy', 'DenseNet121', 'ResNet18', 'swin_s'],            

             }

In [12]:
# for surogate, others in other_list.items():
    
#     true_labels= torch.load(f'./adv_samples/{attack}/cifar10/{surogate}_labels.ckpt')
#     adv_inputs = torch.load(f'./adv_samples/{attack}/cifar10/{surogate}_samples.ckpt')
#     sample_index = torch.load(f'./adv_samples/{attack}/cifar10/{surogate}_index.ckpt')
    
#     adv_dataset = torch.utils.data.TensorDataset(adv_inputs, true_labels, sample_index)
#     adv_loader = torch.utils.data.DataLoader(adv_dataset, batch_size=64,
#                                              shuffle=False, num_workers=4)
#     print(f'Testing model {surogate}')
#     results_df = {}
#     for test_model_name in others:
        
#         results_df[f'{test_model_name}_pred_label'] = []
#         results_df[f'{test_model_name}_pred_conf'] = []
#         results_df[f'{test_model_name}_true_conf'] = []
#         results_df[f'{test_model_name}_true_label'] = []
        
#         param = params[test_model_name]
#         path = model_paths[test_model_name]
#         model = models[test_model_name]
        
#         m = models[test_model_name](**params[test_model_name]).to(device)
#         m.load_state_dict(torch.load(model_paths[test_model_name])['net'])
#         print(f'loaded {path}')
#         m.eval()

#         true_labels = []
#         adv_inputs = []
#         with torch.no_grad():
#             for batch, labels, _ in adv_loader:
#                 batch, labels = batch.to(device), labels.to(device)
#                 outputs = torch.softmax(m(batch), dim=1)
#                 max_return = outputs.max(dim=1)
                
#                 results_df[f'{test_model_name}_pred_label'].extend(max_return.indices.cpu().numpy().tolist())
#                 results_df[f'{test_model_name}_pred_conf'].extend(max_return.values.cpu().numpy().tolist())
                
#                 true_conf = torch.gather(outputs, dim=1, index=labels.view(-1, 1)).view(-1)
#                 results_df[f'{test_model_name}_true_conf'].extend(true_conf.cpu().numpy().tolist())
#                 results_df[f'{test_model_name}_true_label'].extend(labels.view(-1).cpu().numpy().tolist())

#         del m
        
#     results_df['sample_index'] = []
#     for _, _, index in adv_loader:
#         results_df['sample_index'].extend(index.numpy().tolist())
        
#     results_df = pd.DataFrame(results_df)
#     results_df.to_csv(f'./results/{attack}/cifar_surogate_{surogate}.csv', index=False)

for surogate, others in other_list.items():        
    for run in range(20):
        true_labels= torch.load(os.path.join(save_path,
                                             f'adv_samples/PGD/cifar/pertubation_ranking/{surogate}_labels_run_{run}.ckpt'))
        adv_inputs = torch.load(os.path.join(save_path,
                                             f'adv_samples/PGD/cifar/pertubation_ranking/{surogate}_samples_run_{run}.ckpt'))
        run_df = {}
        print(f'Testing model {surogate} | run {run}')
        for test_model_name in others:

            m = models[test_model_name](**params[test_model_name]).to(device)
            m.load_state_dict(torch.load(model_paths[test_model_name])['net'])
            m.eval()

            batch = adv_inputs.to(device)
            with torch.no_grad():
                outputs = torch.softmax(m(batch), dim=1).cpu()
            max_return = outputs.max(dim=1)

            run_df[f'{test_model_name}_pred_label'] = (max_return.indices.numpy().tolist())
            run_df[f'{test_model_name}_pred_conf'] = max_return.values.numpy()

            true_conf = torch.gather(outputs, dim=1, index=true_labels.view(-1, 1)).view(-1)
            run_df[f'{test_model_name}_true_conf'] = (true_conf.cpu().numpy().tolist())
            run_df[f'{test_model_name}_true_label'] = (true_labels.numpy().tolist())

            del m
        run_df = pd.DataFrame(run_df)
        run_df.to_csv(f'./results/pertubation_ranking/cifar/run_{run}_cifar_surogate_{surogate}.csv', index=False)

Testing model ResNet18 | run 0
Testing model ResNet18 | run 1
Testing model ResNet18 | run 2
Testing model ResNet18 | run 3
Testing model ResNet18 | run 4
Testing model ResNet18 | run 5
Testing model ResNet18 | run 6
Testing model ResNet18 | run 7
Testing model ResNet18 | run 8
Testing model ResNet18 | run 9
Testing model ResNet18 | run 10
Testing model ResNet18 | run 11
Testing model ResNet18 | run 12
Testing model ResNet18 | run 13
Testing model ResNet18 | run 14
Testing model ResNet18 | run 15
Testing model ResNet18 | run 16
Testing model ResNet18 | run 17
Testing model ResNet18 | run 18
Testing model ResNet18 | run 19
Testing model swin_s | run 0
Testing model swin_s | run 1
Testing model swin_s | run 2
Testing model swin_s | run 3
Testing model swin_s | run 4
Testing model swin_s | run 5
Testing model swin_s | run 6
Testing model swin_s | run 7
Testing model swin_s | run 8
Testing model swin_s | run 9
Testing model swin_s | run 10
Testing model swin_s | run 11
Testing model swin_s

In [9]:
# df = pd.read_csv('./results/cifar_surogate_DenseNet121.csv')
# other_models = other_list['DenseNet121']

In [10]:
# for o_m in other_models:
#     df[f'{o_m}_transfer'] = (df[f'{o_m}_pred_label'] != df[f'{o_m}_true_label'])

In [11]:
# for i, row in df.iterrows():
#     print(row[['ViT_transfer', 'EfficientNetB0_transfer', 'ResNet18_transfer', 'swin_s_transfer']].mean())

In [12]:
# state = torch.load('./models/cifar10/Swin_1.ckpt')

In [13]:
# m = swin_s()
# m.load_state_dict(state['net'])

In [14]:
# m.mlp_head[1]

In [15]:
# weight = m.mlp_head[1].weight[:10, :]
# bias = m.mlp_head[1].bias[:10]

In [16]:
# m.mlp_head[1] = nn.Linear(768, 10)
# m.mlp_head[1].weight.data = weight
# m.mlp_head[1].bias.data = bias

In [17]:
# state['net'] = m.state_dict()

In [18]:
# torch.save(state, './models/cifar10/Swin_1.ckpt')

In [11]:
df = {}

batch, labels = next(iter(test_loader))
batch, labels = batch.to(device), labels.to(device)

for model_name in model_names:
    if '_copy'  in model_name:
        continue
    m = models[model_name](**params[model_name]).to(device)
    m.load_state_dict(torch.load(model_paths[model_name])['net'])
    print(f'loaded {model_paths[model_name]}')
    m.eval()
    df[f'{model_name}_probs'] = []
    df[f'{model_name}_noise_probs'] = []
    with torch.no_grad():
        sample_num = 0

        probs = torch.softmax(m(batch), dim=1).max(dim=1).values
        noise_probs =torch.softmax(m(torch.clamp(batch+(16/255)*torch.randn_like(batch), 0, 1)),
                                   dim=1).max(dim=1).values

        df[f'{model_name}_probs'] = probs.detach().cpu().numpy().tolist()
        df[f'{model_name}_noise_probs'] = noise_probs.detach().cpu().numpy().tolist()

df = pd.DataFrame(df)
df.to_csv(f'./results/pertubation_ranking/cifar/cifar_all_preds.csv', index=False)
    
     

loaded ./models/cifar10/ViT_0.ckpt
loaded ./models/cifar10/DenseNet121_0.ckpt
loaded ./models/cifar10/EfficientNetB0_0.ckpt
loaded ./models/cifar10/resnet18_0.ckpt
loaded ./models/cifar10/Swin_0.ckpt


In [20]:
import torchvision.transforms as transforms
import cv2

class Resize(object):
    def __init__(self, org_size, inter_size):
        self.org_size = org_size
        self.inter_size = inter_size

    def __call__(self, img):
        img = transforms.functional.resize(img, self.inter_size)
        return transforms.functional.resize(img, self.org_size)
    
class Compress(object):
    def __init__(self,):
        self.encode_param =  [int(cv2.IMWRITE_JPEG_QUALITY), 90]

    def __call__(self, img):
        img = (img.cpu().numpy()*255).astype(np.uint8).transpose([1,2,0])
        result, encimg = cv2.imencode('.jpg', img, self.encode_param)
        decimg = cv2.imdecode(encimg, 1)
        return torch.Tensor(decimg.transpose(2,0,1))/255
    
class Adv_dataset_with_transforms(Dataset):
    def __init__(self, adv_inputs, true_labels, sample_index, transforms):
        self.data = adv_inputs
        self.target = true_labels
        self.sample_index = sample_index
        self.transform = transforms
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index]
        x = self.transform(x)
        
        y = self.target[index]
        sample_index = self.sample_index[index]
        return x, y, sample_index

In [21]:
transform = 'resize'

if transform == 'compression':
    trans = Compress()
elif transform == 'resize':
    trans = Resize(32, 29)
elif transform == 'combined':
    trans = transforms.Compose([
        Compress(),
        Resize(32, 29)
    ])
else:
    trans = lambda x: x

In [22]:
for surogate, others in other_list.items():
    
    true_labels= torch.load(f'./adv_samples/PGD/cifar10/{surogate}_labels.ckpt')
    adv_inputs = torch.load(f'./adv_samples/PGD/cifar10/{surogate}_samples.ckpt')
    sample_index = torch.load(f'./adv_samples/PGD/cifar10/{surogate}_index.ckpt')
    
    adv_dataset = Adv_dataset_with_transforms(adv_inputs, true_labels, sample_index, trans)
    adv_loader = torch.utils.data.DataLoader(adv_dataset, batch_size=64,
                                             shuffle=False, num_workers=4)
    print(f'Testing model {surogate}')
    results_df = {}
    for test_model_name in others:
        
        results_df[f'{test_model_name}_pred_label'] = []
        results_df[f'{test_model_name}_pred_conf'] = []
        results_df[f'{test_model_name}_true_conf'] = []
        results_df[f'{test_model_name}_true_label'] = []
        
        param = params[test_model_name]
        path = model_paths[test_model_name]
        model = models[test_model_name]
        
        m = models[test_model_name](**params[test_model_name]).to(device)
        m.load_state_dict(torch.load(model_paths[test_model_name])['net'])
        print(f'loaded {path}')
        m.eval()

        true_labels = []
        adv_inputs = []
        with torch.no_grad():
            for batch, labels, _ in adv_loader:
                batch, labels = batch.to(device), labels.to(device)
                outputs = torch.softmax(m(batch), dim=1)
                max_return = outputs.max(dim=1)
                
                results_df[f'{test_model_name}_pred_label'].extend(max_return.indices.cpu().numpy().tolist())
                results_df[f'{test_model_name}_pred_conf'].extend(max_return.values.cpu().numpy().tolist())
                
                true_conf = torch.gather(outputs, dim=1, index=labels.view(-1, 1)).view(-1)
                results_df[f'{test_model_name}_true_conf'].extend(true_conf.cpu().numpy().tolist())
                results_df[f'{test_model_name}_true_label'].extend(labels.view(-1).cpu().numpy().tolist())

        del m
        
    results_df['sample_index'] = []
    for _, _, index in adv_loader:
        results_df['sample_index'].extend(index.numpy().tolist())
        
    results_df = pd.DataFrame(results_df)
    results_df.to_csv(f'./results/transforms/{transform}/cifar_surogate_{surogate}.csv', index=False)

Testing model ResNet18
loaded ./models/cifar10/ViT_0.ckpt
loaded ./models/cifar10/resnet18_1.ckpt
loaded ./models/cifar10/DenseNet121_0.ckpt
loaded ./models/cifar10/EfficientNetB0_0.ckpt
loaded ./models/cifar10/Swin_0.ckpt
Testing model swin_s
loaded ./models/cifar10/ViT_0.ckpt
loaded ./models/cifar10/Swin_1.ckpt
loaded ./models/cifar10/DenseNet121_0.ckpt
loaded ./models/cifar10/EfficientNetB0_0.ckpt
loaded ./models/cifar10/resnet18_0.ckpt
Testing model ViT
loaded ./models/cifar10/DenseNet121_0.ckpt
loaded ./models/cifar10/ViT_1.ckpt
loaded ./models/cifar10/EfficientNetB0_0.ckpt
loaded ./models/cifar10/resnet18_0.ckpt
loaded ./models/cifar10/Swin_0.ckpt
Testing model DenseNet121
loaded ./models/cifar10/ViT_0.ckpt
loaded ./models/cifar10/DenseNet121_1.ckpt
loaded ./models/cifar10/EfficientNetB0_0.ckpt
loaded ./models/cifar10/resnet18_0.ckpt
loaded ./models/cifar10/Swin_0.ckpt
Testing model EfficientNetB0
loaded ./models/cifar10/ViT_0.ckpt
loaded ./models/cifar10/EfficientNetB0_1.ckpt
lo